Работу выполнили: Энзе Гонг и Катричева Надежда

## 1. Подготовка данных

In [1]:
import pandas as pd
import csv

Open train, change NaN for 0.

In [2]:
train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE,encoding='utf-8').fillna(0)
len(train)

16406

In [3]:
train.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,0,0,0,0,0,0
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
2,"Заместитель Генерального секретаря подчеркнул,...",0,0,0,0,0,0,0
3,Продажа недвижимости из собственных портфелей ...,0,0,0,0,0,0,0
4,"Новым является то, что повышенное давление кон...",0,0,0,0,0,0,0


The class-named column hinders using some pandas methods, so let's change the column name.

In [13]:
train.rename(index=str, columns={"class": "class_"}, inplace=True,) # inplace=True means the df is really changed
train.head()

,text,class_,cV,cR1,cR2,V,R1,R2
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,0,0,0,0,0,0
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
2,"Заместитель Генерального секретаря подчеркнул,...",0,0,0,0,0,0,0
3,Продажа недвижимости из собственных портфелей ...,0,0,0,0,0,0,0
4,"Новым является то, что повышенное давление кон...",0,0,0,0,0,0,0


Classes 0 and 1 in train.csv

In [7]:
train.query('class_ == "0"').class_.count() #same (train.class_ == 0).sum()

10864

In [8]:
train.query('class_ == "1"').class_.count() #same (train.class_ == 1).sum()

5542

Classes 0 and 1 in dev.csv

In [10]:
dev = pd.read_csv('dev.csv',sep = '\t', quoting=csv.QUOTE_NONE).fillna(0)

dev.rename(index=str, columns={"class": "class_"}, inplace=True,)
#dev.head()
(dev.class_ == 0).sum()

2760

In [11]:
(dev.class_ == 1).sum()

1382

Classes 0 and 1 in test.csv

In [76]:
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE).fillna(0)

test.rename(index=str, columns={"class": "class_"}, inplace=True,)
#test.head()
(test.class_ == 0).sum()

1365

In [95]:
(test.class_ == 1).sum()

680

## 2. Анализ разметки

In [14]:
"""We could view only class 1, but it's not necessary"""
#markup = train.loc[train['class_'] == 1]
#markup.head()

"""Drop what's not the ellipsis offset markup cells""" 
stat_train = train.drop(['text', 'class_'], axis=1)

stat_train.head()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,92:99,83:91,103:109,127:127,119:124,127:134
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [15]:
"""Change offsets for the number of offsets in a cess"""

labels = ['cV', 'cR1', 'cR2', 'V', 'R1', 'R2']

for i in stat_train.index:
    for label in labels:
        if stat_train.loc[i, label] != 0:
            stat_train.loc[i, label] = len((stat_train.loc[i, label]).split()) # offsets are split by spaces 

stat_train.head()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [16]:
"""To see the unique rows"""
stat_train.drop_duplicates()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
14,1,1,0,1,1,0
117,1,1,1,2,2,2
211,1,1,1,3,3,3
2285,1,1,1,4,4,4
5902,1,1,1,5,5,5
8104,1,0,1,1,1,1


We can see that there are 8 different configurations in test.


To see the numbers of sentences of such configurations we'll use groupby, numbers on the right.

In [25]:
stat_train.groupby(stat_train.columns.tolist(),as_index=False).size()

cV  cR1  cR2  V  R1  R2
0   0    0    0  0   0     10864
1   0    1    1  1   1         1
    1    0    1  1   0        77
         1    1  1   1      5095
              2  2   2       315
              3  3   3        45
              4  4   4         7
              5  5   5         2
dtype: int64

Do the same to the other datasets.


Dev

In [33]:
stat_dev = dev.drop(['text', 'class_'], axis=1)

labels = ['cV', 'cR1', 'cR2', 'V', 'R1', 'R2']

for i in stat_dev.index:
    for label in labels:
        if stat_dev.loc[i, label] != 0:
            stat_dev.loc[i, label] = len((stat_dev.loc[i, label]).split())

stat_dev.drop_duplicates()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
61,1,1,1,2,2,2
297,1,1,0,1,1,0
355,1,1,1,3,3,3
888,1,1,1,7,7,7
938,1,1,1,4,4,4


7 different cases in dev, with such ststistics:

In [26]:
stat_dev.groupby(stat_dev.columns.tolist(),as_index=False).size()

cV  cR1  cR2  V  R1  R2
0   0    0    0  0   0     2760
1   1    0    1  1   0       27
         1    1  1   1     1265
              2  2   2       73
              3  3   3        9
              4  4   4        7
              7  7   7        1
dtype: int64

Test

In [31]:
stat_test = test.drop(['text', 'class_'], axis=1)

labels = ['cV', 'cR1', 'cR2', 'V', 'R1', 'R2']

for i in stat_test.index:
    for label in labels:
        if stat_test.loc[i, label] != 0:
            stat_test.loc[i, label] = len((stat_test.loc[i, label]).split())

stat_test.drop_duplicates()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
23,1,1,1,2,2,2
254,1,1,0,1,1,0
600,1,1,1,5,5,5
758,1,1,1,3,3,3
1115,1,1,1,4,4,4
1696,1,1,0,2,2,0


8 different cases in dev, with such ststistics:

In [30]:
stat_test.groupby(stat_test.columns.tolist(),as_index=False).size()

cV  cR1  cR2  V  R1  R2
0   0    0    0  0   0     1365
1   1    0    1  1   0       16
              2  2   0        1
         1    1  1   1      616
              2  2   2       42
              3  3   3        3
              4  4   4        1
              5  5   5        1
dtype: int64

# 3. Конвертер разметки-1

In [233]:
class_1 = train.loc[train['class_'] == 1] #The rows with gapping

In [53]:
class_1.head()

,text,class_,cV,cR1,cR2,V,R1,R2
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
5,"Выворачивает наизнанку, запад превращает в юг,...",1,30:40,24:29,41:45,57:57,49:54,57:65
7,Текущие обязательства компании в 2009 г. сокра...,1,109:120,82:108,121:148,180:180,152:177,180:202
8,Первая конвенция была ратифицирована 80 процен...,1,22:36,0:16,37:74,87:87,78:84,87:116
14,"Но Христианин сказал Упрямому: — Нет, сосед, л...",1,105:112,113:139,0,156:156,156:169,0


In [50]:
def str_to_inx(inx_str):
    '''
    Change slices from strings into integers.
    '''
    inx_str = str(inx_str)
    inx_str = inx_str.split(' ')
    inx_list = []
    for i in inx_str:
        i = i.split(':')
        for inx in i:
            inx_list.append(inx)
    
    return inx_list
        

def IOB_encoder(df):
    '''
    Split texts into words, and map them with their IOB properties.
    '''
    encoded = []
    for i in range(len(df)):
        text = df['text'].iloc[i]
        cv_inx = str_to_inx(df['cV'].iloc[i])
        cR1_inx = str_to_inx(df['cR1'].iloc[i])
        cR2_inx = str_to_inx(df['cR2'].iloc[i])
        V_inx = str_to_inx(df['V'].iloc[i])
        R1_inx = str_to_inx(df['R1'].iloc[i])
        R2_inx = str_to_inx(df['R2'].iloc[i])
        
        cv = [text[int(cv_inx[int(2*n)]):int(cv_inx[int(2*n+1)])] for n in range(int(len(cv_inx)/2))]
        #print(cv)
        cR1 = [text[int(cR1_inx[int(2*n)]):int(cR1_inx[int(2*n+1)])] for n in range(int(len(cR1_inx)/2))]
        cR2 = [text[int(cR2_inx[int(2*n)]):int(cR2_inx[int(2*n+1)])] for n in range(int(len(cR2_inx)/2))]
        V = [text[int(V_inx[int(2*n)]):int(V_inx[int(2*n+1)])] for n in range(int(len(V_inx)/2))]
        R1 = [text[int(R1_inx[int(2*n)]):int(R1_inx[int(2*n+1)])] for n in range(int(len(R1_inx)/2))]
        R2 = [text[int(R2_inx[int(2*n)]):int(R2_inx[int(2*n+1)])] for n in range(int(len(R2_inx)/2))]
        
        IOB = {}
        
        if len(cv) > 0:
            for k in cv:
                text = text.replace(k,'')
                IOB[k] = 'B-cV'
        if len(cR1) > 0:
            for k in cR1:
                text = text.replace(k,'')
                IOB[k] = 'B-cR1'
        if len(cR2) > 0:
            for k in cR2:
                text = text.replace(k,'')
                words = cR2[0].split(' ')
                if len(words) > 1:
                    IOB[words[0]] = 'B-cR2'
                    IOB[' '.join(words[1:])] = 'I-cR2'
                elif len(words) == 1:
                    IOB[words[0]] = 'B-cR2'
                
        if len(V) > 0:
            for k in V:
                text = text.replace(k,'')
                IOB[k] = 'V'
        if len(R1) > 0:
            for k in R1:
                text = text.replace(k,'')
                IOB[k] = 'B-R1'
        if len(R2) > 0:
            for j in range(len(R2)):
                text = text.replace(R2[j],'')
                words = R2[j].split(' ')
                
                if len(words) > 1:
                    IOB[words[0]] = 'B-R2'
                    IOB[' '.join(words[1:])] = 'I-R2'
                else:
                    IOB[words[0]] = 'B-R2'
        
        text = text.split(' ')
        
        for word in text:
            if word not in IOB:
                IOB[word] = 'O'
            
        
        encoded.append(IOB)
        
        
    
    return encoded
    
        
def IOB_decoder(encoded):
    '''
    Turn IOB into slices. Since V class is non-existent, the exact slices generated my differ from the original ones. 
    '''
    decoded = []
    
    for i in range(len(encoded)):
        text = class_1['text'].iloc[i]
        IOB = {}
        for word in encoded[i]:
            if encoded[i][word] == 'B-cV':
                cV_start = text.index(word)
                cV_end = cV_start + len(word)
                IOB['cV'] = str(cV_start) + ':' + str(cV_end)
            elif encoded[i][word] == 'B-cR1':
                cR1_start = text.index(word)
                cR1_end = cV_start + len(word)
                IOB['cR1'] = str(cR1_start) + ':' + str(cR1_end)
            elif encoded[i][word] == 'V':
                V_start = text.index(word)
                V_end = V_start + len(word)
                IOB['V'] = str(V_start) + ':' + str(V_end)
            elif encoded[i][word] == 'B-R1':
                R1_start = text.index(word)
                R1_end = R1_start + len(word)
                IOB['R1'] = str(R1_start) + ':' + str(R1_end)
            elif encoded[i][word] == 'B-cR2':
                cR2_start = text.index(word)
                if 'I-cR2' in encoded[i].values():
                    for word2 in encoded[i]:
                        if encoded[i][word2] == 'I-cR2':
                            cR2_end = text.index(word2) + len(word2)
                    IOB['cR2'] = str(cR2_start) + ':' + str(cR2_end)
                else:
                    cR2_end = text.index(word) + len(word)
                    IOB['cR2'] = str(cR2_start) + ':' + str(cR2_end)
            elif encoded[i][word] == 'B-R2':
                R2_start = text.index(word)
                if 'I-R2' in encoded[i].values():
                    for word2 in encoded[i]:
                        if encoded[i][word2] == 'I-R2':
                            R2_end = text.index(word2) + len(word2)
                    IOB['R2'] = str(R2_start) + ':' + str(R2_end)
                else:
                    R2_end = text.index(word) + len(word)
                    IOB['R2'] = str(R2_start) + ':' + str(R2_end)
        
        decoded.append(IOB)
        
    return decoded

In [51]:
encoded = IOB_encoder(class_1)

In [57]:
encoded[0]

{'состоит': 'B-cV',
 'часть их': 'B-cR1',
 'одного': 'B-cR2',
 '': 'V',
 'часть': 'B-R1',
 'из': 'B-R2',
 'двух': 'I-R2',
 'Работа': 'O',
 'с': 'O',
 'двухбайтовыми': 'O',
 'наборами': 'O',
 'символов': 'O',
 '—': 'O',
 'просто': 'O',
 'кошмар': 'O',
 'для': 'O',
 'программиста,': 'O',
 'так': 'O',
 'как': 'O',
 'байта,': 'O',
 'а': 'O',
 '.': 'O'}

In [47]:
decoded = IOB_decoder(encoded)

In [58]:
decoded[0]

{'cV': '92:99',
 'cR1': '83:100',
 'cR2': '103:109',
 'V': '0:0',
 'R1': '83:88',
 'R2': '100:13'}

## 4. Конвертер разметки-2

In [456]:
def str_to_inx(inx_str):
    '''
    Change slices from strings into integers.
    '''
    inx_str = str(inx_str)
    inx_str = inx_str.split(' ')
    inx_list = []
    for i in inx_str:
        i = i.split(':')
        for inx in i:
            inx_list.append(inx)
    return inx_list


def bracket_encoder(df):
    '''
    Get labelled chunks, get all the chunks, combine the labelled and general chunks into sentences.
    '''
  #  encoded = {}
    enc_sent = []
  #  num_sent = []
    for i in range(len(df)):
        print(i)
        text = df['text'].iloc[i]
        print(text)
        cv_inx = str_to_inx(df['cV'].iloc[i])
        cR1_inx = str_to_inx(df['cR1'].iloc[i])
        cR2_inx = str_to_inx(df['cR2'].iloc[i])
        V_inx = str_to_inx(df['V'].iloc[i])
        R1_inx = str_to_inx(df['R1'].iloc[i])
        R2_inx = str_to_inx(df['R2'].iloc[i])
        
        all_inx = ''
        all_inx = cv_inx+cR1_inx+cR2_inx+V_inx+R1_inx+R2_inx # get offset numbers for the whole sent
        
        sort_all_ind = [int(inx) for inx in all_inx if inx != '0'] # str -> int, rid of zeros
        sort_all_ind.append(0) # add the only needed zero
        sort_all_ind.sort()
        sort_all_ind.append(len(text) + 2) # needed to get the index of the last symbol in a sentence
        sort_all_ind.append(len(text) + 2) # to indeed have this index in the list after the latter manipulations
        all_inx_pairs = [sort_all_ind[i:i + 2] for i in range(len(sort_all_ind) - 3 + 1) if sort_all_ind[i] != sort_all_ind[i+2]]
        all_symb = [] # collect all the parts of the sent
        for pair in all_inx_pairs:
            text_pair = text[pair[0]:pair[1]]
            all_symb.append(text_pair)
        
        cv = [text[int(cv_inx[int(2*n)]):int(cv_inx[int(2*n+1)])] for n in range(int(len(cv_inx)/2))]
        cR1 = [text[int(cR1_inx[int(2*n)]):int(cR1_inx[int(2*n+1)])] for n in range(int(len(cR1_inx)/2))]
        cR2 = [text[int(cR2_inx[int(2*n)]):int(cR2_inx[int(2*n+1)])] for n in range(int(len(cR2_inx)/2))]
        V = [text[int(V_inx[int(2*n)]):int(V_inx[int(2*n+1)])] for n in range(int(len(V_inx)/2))]
        R1 = [text[int(R1_inx[int(2*n)]):int(R1_inx[int(2*n+1)])] for n in range(int(len(R1_inx)/2))]
        R2 = [text[int(R2_inx[int(2*n)]):int(R2_inx[int(2*n+1)])] for n in range(int(len(R2_inx)/2))]
        ind_R2 = [[int(R2_inx[int(2*n)]), int(R2_inx[int(2*n+1)])] for n in range(int(len(R2_inx)/2))]
        
        enc_s = []
        for s in all_symb:
            for k in cv:
                if s == k:
                    s = 'cv' + '[ ' + s + ' cv' + ']'
            for k in cR1:
                if s == k:
                    s = 'cR1' + '[ ' + s + ' cR1' + ']'
            for k in cR2:
                if s == k:
                    s = 'cR2' + '[ ' + s + ' cR2' + ']'
            for k in V:
                if s == k and s == '':
                    s = 'V' + '[] '
                elif s == k and s != '':
                    s = 'V' + '[ ' + s + ' V' + ']'
            for k in R1:
                if s == k and s == '':
                    s = 'R1' + '[] '
                elif s == k and s != '':
                    s = 'R1' + '[ ' + s + ' R1' + ']'
            for k in R2:
                if s == k and s == '':
                    s = 'R2' + '[] '
                elif s == k and s != '':
                    s = 'R2' + '[ ' + s + ' R2' + ']'
            enc_s.append(s)
        print(''.join(enc_s))
        enc_sent.append(''.join(enc_s))
    #    num_sent.append(i)
  #  print(num_sent, enc_sent)
  #  encoded = dict(zip(num_sent, enc_sent))    
    print(encoded)
    return(encoded)
# ''.join(encoded)


In [457]:
print(bracket_encoder(class_1))

0
Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.
Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как cR1[ часть их cR1] cv[ состоит cv] из cR2[ одного cR2] байта, а R1[ часть R1] — V[] R2[ из двух R2].
1
Выворачивает наизнанку, запад превращает в юг, а север — в восток, меняет местами добро и зло, велит открыть своё сердце, ничего не требуя взамен.
Выворачивает наизнанку, cR1[ запад cR1] cv[ превращает cv] cR2[ в юг cR2], а R1[ север R1] — V[] R2[ в восток R2], меняет местами добро и зло, велит открыть своё сердце, ничего не требуя взамен.
2
Текущие обязательства компании в 2009 г. сократились на 16% - до 135.3 млн. грн., долгосрочные обязательства уменьшились на 1.7% - до 71.8 млн. грн., а дебиторская задолженность - на 3% до 87.7 млн. грн.
Текущие обязательства компании в 2009 г. сократились на 16% - до 135.3 млн. грн., cR1[ долгосрочные обязательства cR1] cv[ умень

326
Эти контакты соприкасаются с одной стороны с контактами устройства, а с другой — с кожей пациента (см. Рисунок 2)
Эти контакты cv[ соприкасаются cv] cR1[ с одной стороны cR1] cR2[ с контактами устройства cR2], а R1[ с другой R1] — V[] R2[ с кожей пациента (см. Рисунок 2) R2]V[] 
327
Традиционный подход к анализу поведения фирмы в рыночной экономике основан на том, что профсоюзы обладают некоторой монопольной властью на рынке труда, а фирма — определенной рыночной властью на рынке производимого ею продукта.
Традиционный подход к анализу поведения фирмы в рыночной экономике основан на том, что cR1[ профсоюзы cR1] cv[ обладают cv] cR2[ некоторой монопольной властью на рынке труда cR2], а R1[ фирма R1] — V[] R2[ определенной рыночной властью на рынке производимого ею продукта R2].
328
По сравнению с 2000 годом, в 2002 году в Украине число детей, рожденных ВИЧ-инфицированными женщинами, увеличилось почти на 90% (в 2000 году таких детей зарегистрировано 737, а в 2002 году — 1379).
По сра

С усилением спроса школы философии и риторики сделались постоянными сперва в Афинах, а потом в ряде других городов.
С усилением спроса школы философии и риторики cv[ сделались cv] постоянными cR1[ сперва cR1] cR2[ в Афинах cR2], а R1[ потом R1] V[] R2[ в ряде других городов R2].
595
На ведущей торговой площадке — ФБ ММВБ средний дневной оборот торгов акциями возрос в 1,9 раза по сравнению с аналогичным показателем декабря, в РТС — почти в 3,5 раза.
cR1[ На ведущей торговой площадке — ФБ ММВБ cR1] средний дневной оборот торгов акциями cv[ возрос cv] cR2[ в 1,9 раза cR2] по сравнению с аналогичным показателем декабря, R1[ в РТС R1] — V[] R2[ почти в 3,5 раза R2].
596
Я работаю переводчиком, супруг техническим инженером.
cR1[ Я cR1] cv[ работаю cv] cR2[ переводчиком cR2], R1[ супруг R1] V[] R2[ техническим инженером R2].
597
Черный откликается на Чарли, а рыжий на Сему.
cR1[ Черный cR1] cv[ откликается cv] cR2[ на Чарли cR2], а R1[ рыжий R1] V[] R2[ на Сему R2].
598
Лицо из бледного сдела

811
Гражданское общество порождает подлинный плюрализм, плюрализм ведет к конкуренции, конкуренция -- к качеству.
Гражданское общество порождает подлинный плюрализм, cR1[ плюрализм cR1] cv[ ведет cv] cR2[ к конкуренции cR2], R1[ конкуренция R1] -- V[] R2[ к качеству R2].
812
В секторе Газа за последние три года население Кфар-Дарома увеличилось на 52 процента, а Нецарима — на 24 процента.
В секторе Газа за последние три года население cR1[ Кфар-Дарома cR1] cv[ увеличилось cv] cR2[ на 52 процента cR2], а R1[ Нецарима R1] — V[] R2[ на 24 процента R2].
813
Одна доля этой р-орбитали перекрывается с нуклеофилом, а вторая — с уходящей группой.
cR1[ Одна доля этой р-орбитали cR1] cv[ перекрывается cv] cR2[ с нуклеофилом cR2], а R1[ вторая R1] — V[] R2[ с уходящей группой R2].
814
В первом из них происходит торможение потока в системе ударных волн до звуковой скорости, а во втором диффузоре до скорости потребной для дальнейшей транспортировки газа.
cR1[ В первом из них cR1] cv[ происходит cv] 

Реальный курс рубля к доллару за год увеличился на 14,8%, а к евро - на 5,7%.
cR1[ Реальный курс рубля к доллару за год cR1] cv[ увеличился cv] cR2[ на 14,8% cR2], а R1[ к евро R1] - V[] R2[ на 5,7% R2].
928
-СТЕРИЛИЗАЦИЯ КОШКИ В ДОМАШНИХ УСЛОВИЯХ - порой единственно приемлемый способ избавить вашу любимицу от страданий, вызванных непрекращающейся течкой, а владельца - от кошкиных бесконечных ночных концертов.
-СТЕРИЛИЗАЦИЯ КОШКИ В ДОМАШНИХ УСЛОВИЯХ - порой единственно приемлемый способ cv[ избавить cv] cR1[ вашу любимицу cR1] cR2[ от страданий, вызванных непрекращающейся течкой cR2], а R1[ владельца R1] - V[] R2[ от кошкиных бесконечных ночных концертов R2].
929
Ткань рубашки была такой тонкой, а ее руки такими горячими!
cR1[ Ткань рубашки cR1] cv[ была cv] cR2[ такой тонкой cR2], а R1[ ее руки R1] V[] R2[ такими горячими R2]!
930
Крупный объем средств ЕБРР пойдет на модернизацию основных подстанций сети электропередач, а небольшая часть ресурсов - на создание Энергетического углеродн

...

Здесь же 6 указано, что аналогичные результаты получены Ивенсом и Иоффе для азида меди, а Бертхольдом — для ацетиленида серебра.
Здесь же 6 указано, что аналогичные результаты cv[ получены cv] cR1[ Ивенсом и Иоффе cR1] cR2[ для азида меди cR2], а R1[ Бертхольдом R1] — V[] R2[ для ацетиленида серебра R2].
4376
Так, снайперам из обоих департаментов предлагалось занять огневые позиции на крышах домов через улицу от мотеля, а штурмовикам стоять наготове и ждать команды к выступлению.
Так, cR1[ снайперам из обоих департаментов cR1] cv[ предлагалось cv] cR2[ занять огневые позиции на крышах домов через улицу от мотеля cR2], а R1[ штурмовикам R1] V[] R2[ стоять наготове и ждать команды к выступлению R2].
4377
Параметры, задаваемые в конфигурационном файле, называются декларативным кодированием, а параметры, задаваемые при помощи исходного С#-кода, — императивним кодированием.
cR1[ Параметры, задаваемые в конфигурационном файле cR1], cv[ называются cv] cR2[ декларативным кодированием cR2], а R

cR1[ Газоснабжение cR1] cv[ подорожало cv] cR2[ на 4,2% cR2], R1[ электроснабжение R1] — V[] R2[ на 4% R2], пишет РИА Новости.
4592
Двигатели постоянного тока для катушечных магнитофонов должны изготовляться на номинальные частоты в диапазоне от 200 до 1000 мин-1 , для кассетных магнитофонов - в диапазоне от 1 000 до 4 000 мин-1
Двигатели постоянного тока cR1[ для катушечных магнитофонов cR1] cv[ должны cv] изготовляться на номинальные частоты cR2[ в диапазоне от 200 до 1000 мин-1 cR2] , R1[ для кассетных магнитофонов R1] - V[] R2[ в диапазоне от 1 000 до 4 000 мин-1 R2]V[] 
4593
По данным Ростуризма, в 2011 г. въездной турпоток в Санкт-Петербург увеличился в среднем на 7-8%, а поток иностранных туристов - примерно на 10%, что обусловлено введением безвизового режима с такими государствами, как Чили, Уругвай.
По данным Ростуризма, в 2011 г. cR1[ въездной турпоток в Санкт-Петербург cR1] cv[ увеличился cv] cR2[ в среднем на 7-8% cR2], а R1[ поток иностранных туристов R1] - V[] R2[ пример

Из этой суммы cR1[ 2861 млн. долл. cR1] cv[ предназначались cv] cR2[ на предметы снабжения, транспорт, доставку и другие прямые расходы на обслуживание cR2], а R1[ 228,9 млн. долл. R1] - V[] R2[ на соответствующий компонент бюджета обеспечения программ/административного бюджета (ОПА) R2].
4869
Секретарем подготовительного комитета был избран француз Луи Сайян, а председателем Уолтер Ситрин.
cR1[ Секретарем подготовительного комитета cR1] был cv[ избран cv] cR2[ француз Луи Сайян cR2], а R1[ председателем R1] V[] R2[ Уолтер Ситрин R2].
4870
Его лицо стало серьезным, а взгляд – безрадостным.
cR1[ Его лицо cR1] cv[ стало cv] cR2[ серьезным cR2], а R1[ взгляд R1] – V[] R2[ безрадостным R2].
4871
Среднее значение частоты несущей для более низкочастотной подзоны составляет 1170 Гц, при этом частоты сигналов смещаются до значения 1270 Гц — для передачи логической единицы и до частоты 1070 Гц — для передачи двоичного нуля.
Среднее значение частоты несущей для более низкочастотной подзоны соста

Было показано, что реакция в начальной стадии имеет нулевой порядок по винилацетилену и первый — по водороду.
Было показано, что реакция в начальной стадии cv[ имеет cv] cR1[ нулевой cR1] порядок cR2[ по винилацетилену cR2] и R1[ первый R1] — V[] R2[ по водороду R2].
5092
Там, где река становится шире, а течение – медленнее.
Там, где cR1[ река cR1] cv[ становится cv] cR2[ шире cR2], а R1[ течение R1] – V[] R2[ медленнее R2].
5093
Байты 8-11 показывают, что в группе имеется 1912 (0x0778) свободных индексных узлов, а байты 12-15 — что в неполных блоках содержится 11 (ОхОЬ) свободных фрагментов.
cR1[ Байты 8-11 cR1] cv[ показывают cv], cR2[ что в группе имеется 1912 (0x0778) свободных индексных узлов cR2], а R1[ байты 12-15 R1] — V[] R2[ что в неполных блоках содержится 11 (ОхОЬ) свободных фрагментов R2].
5094
Отцом Гильгамеша был верховный жрец Урука, а матерью — богиня Нинсун.
cR1[ Отцом Гильгамеша cR1] cv[ был cv] верховный жрец cR2[ Урука cR2], а R1[ матерью R1] — V[] R2[ богиня Нинсу

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 5. Бинарная классификация

In [60]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from nltk import word_tokenize

Using TensorFlow backend.


In [62]:
def clear_text(df, MAX_length):
    '''
    Decapitalize the text and remove punctuations, numbers, etc.. Also remove sentences that are too long.
    '''
    docs = []
    labels = []
    IOB = []
    text_index = []
    for i in range(len(df)):
        current_pair = []
        sent = df['text'][i]
        sent = sent.lower()
        ru = ' йцукенгшщзхъфывапролджэячсмитьбюё'
        sent = ''.join([letter for letter in sent if letter in ru])
        if len(sent.split(' ')) <= MAX_length:
            docs.append(sent)
            labels.append(df['class_'][i])
            text_index.append(i)
    labels = array(labels)
    return docs, labels, text_index

docs_train, labels_train, text_index = clear_text(train, 50)

In [63]:
def make_vocab(docs):
    '''
    Make a vocabulary for later use.
    '''
    vocab = []
    for i in docs:
        i = i.split(' ')
        for word in i:
            if word not in vocab:
                vocab.append(word)
    
    return vocab

vocab = make_vocab(docs_train)

In [64]:
len(vocab)

63616

In [65]:
docs_train[0:2]

['будучи в прошлый четверг в софии он назвал себя наполовину восточноевропейцем и за последние пять дней успел принять у себя лидеров украины польши и чехии',
 'работа с двухбайтовыми наборами символов  просто кошмар для программиста так как часть их состоит из одного байта а часть  из двух']

In [66]:
vocab_size = len(vocab)
encoded_docs = [one_hot(d, vocab_size) for d in docs_train]#turn sentences from strings into arrays of indexes.

In [68]:
encoded_docs[0:5]

[[49886,
  60558,
  17844,
  56856,
  60558,
  60552,
  56598,
  42147,
  61513,
  8200,
  44424,
  9452,
  43860,
  15979,
  23458,
  41756,
  15782,
  29215,
  52627,
  61513,
  30956,
  3555,
  23001,
  9452,
  54317],
 [35387,
  48962,
  25945,
  48765,
  53016,
  57747,
  61822,
  23224,
  41010,
  57293,
  30894,
  49067,
  32921,
  55973,
  60613,
  56847,
  27042,
  57101,
  49067,
  60613,
  7997],
 [33806,
  61119,
  46142,
  33223,
  59240,
  18080,
  28248,
  45972,
  53989,
  53448,
  26939,
  60558,
  9866,
  16155,
  9452,
  31533,
  32959,
  12434,
  58800,
  9452,
  55040,
  38005,
  40713,
  25850,
  9452,
  2086,
  35600,
  32877,
  58537,
  19891,
  27674,
  61629,
  48459,
  14013,
  39063,
  41930,
  9452,
  56860],
 [31759,
  19218,
  60613,
  63339,
  4491,
  26898,
  39252,
  40724,
  60558,
  51549,
  56847,
  60613,
  47410,
  57101,
  63585,
  42279,
  4606,
  19132,
  23224,
  8836,
  60904,
  22551,
  41346],
 [53115,
  58271,
  42688,
  5383,
  60558,
  5

In [69]:
MAX_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_length, padding='post') #Make all sentences of the same length.
print(padded_docs)

[[49886 60558 17844 ...     0     0     0]
 [35387 48962 25945 ...     0     0     0]
 [33806 61119 46142 ...     0     0     0]
 ...
 [  959 43162 39746 ...     0     0     0]
 [15412 48962  3059 ...     0     0     0]
 [24111 57023 27560 ...     0     0     0]]


Now we build and train a model.

In [70]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=MAX_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           6361600   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5001      
Total params: 6,366,601
Trainable params: 6,366,601
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
model.fit(padded_docs, labels_train, epochs=100, verbose=0)

The training takes forever. So please load the model included in the directory.

In [84]:
from keras.models import load_model
model = load_model('task5__.model')

# 6. Оценка результатов бинарной классификации

The next step is to make the files for testing.

In [85]:
#test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE,encoding='utf-8').fillna(0)

In [86]:
docs_test, labels_test, text_index_test = clear_text(test, 50)
encoded_test = [one_hot(d, vocab_size) for d in docs_test]
padded_test = pad_sequences(encoded_test, maxlen=MAX_length, padding='post')

In [87]:
loss, accuracy = model.evaluate(padded_test, labels_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 55.466399


The achieved accuracy is 55.4%.

Now let's use the script. To do it, we have to prepare the input dfs and then csvs, as we have got rid of the sentences of more than 50 words.

In [147]:
zeros = []
for i in enumerate(docs_test):
    zeros.append('0')

In [206]:

test_ = pd.DataFrame({'text': docs_test, 'class_': labels_test, 'cV':zeros, 'cR1':zeros, 'cR2':zeros, 'V':zeros, 'R1':zeros, 'R2':zeros})

In [207]:
test_.to_csv(r'test_.csv', sep='\t')

Get predicted class labels:

In [208]:
prediction = model.predict(padded_test)
predi_cate = []
for i in prediction:
    if i < 0.5:
        predi_cate.append(0)
    else:
        predi_cate.append(1)

In [209]:
output = test_

In [210]:
test_.head(15)

,text,class_,cV,cR1,cR2,V,R1,R2
0,изобретение относится к судостроению и касаетс...,0,0,0,0,0,0,0
1,эти состояния называют фазами воды а превращен...,1,0,0,0,0,0,0
2,и должен ни единой долькой не отступаться от ...,0,0,0,0,0,0,0
3,он потребовал обеспечить полное осуществление ...,0,0,0,0,0,0,0
4,по мнению местного пастора элла эбанкса запрет...,0,0,0,0,0,0,0
5,международные правозащитники некоторые национа...,0,0,0,0,0,0,0
6,концы элементов закреплены в подвижных трубных...,0,0,0,0,0,0,0
7,два валяются в воде два на призовых местах,1,0,0,0,0,0,0
8,в тоже время в обоих двоичных деревьях узел я...,1,0,0,0,0,0,0
9,знаете обычно говорят что девочка это папина ...,0,0,0,0,0,0,0


In [211]:
output.drop(['class_'], axis = 1, inplace=True)
output.head()

,text,cV,cR1,cR2,V,R1,R2
0,изобретение относится к судостроению и касаетс...,0,0,0,0,0,0
1,эти состояния называют фазами воды а превращен...,0,0,0,0,0,0
2,и должен ни единой долькой не отступаться от ...,0,0,0,0,0,0
3,он потребовал обеспечить полное осуществление ...,0,0,0,0,0,0
4,по мнению местного пастора элла эбанкса запрет...,0,0,0,0,0,0


In [212]:
output.insert(1, "class_", predi_cate, True)

In [213]:
output.head()

,text,class_,cV,cR1,cR2,V,R1,R2
0,изобретение относится к судостроению и касаетс...,0,0,0,0,0,0,0
1,эти состояния называют фазами воды а превращен...,0,0,0,0,0,0,0
2,и должен ни единой долькой не отступаться от ...,1,0,0,0,0,0,0
3,он потребовал обеспечить полное осуществление ...,0,0,0,0,0,0,0
4,по мнению местного пастора элла эбанкса запрет...,0,0,0,0,0,0,0


In [214]:
output.to_csv(r'output.csv', sep='\t')

In [215]:
! python3 agrr_metrics.py -b test_.csv output.csv

Binary classification quality (f1-score): 0.24489795918367344
Other metrics: 
 Precision: 0.28628230616302186
 Recall: 0.2139673105497771


Well, just awful. That must be the result of too much of normalization and lack of postags, so we'll try to prepare data differently.

# 7. Классификация последовательности

In [78]:
train_encoded = IOB_encoder(train)

In [80]:
to_filter = [train_encoded[i] for i in range(len(train))if i not in text_index]#Filter from train_encoded those sentences that are too long.
for sent in to_filter:
    train_encoded.remove(sent)

In [82]:
train_encoded

[{'Будучи': 'O',
  'в': 'O',
  'прошлый': 'O',
  'четверг': 'O',
  'Софии,': 'O',
  'он': 'O',
  'назвал': 'O',
  'себя': 'O',
  "'наполовину": 'O',
  "восточноевропейцем',": 'O',
  'и': 'O',
  'за': 'O',
  'последние': 'O',
  'пять': 'O',
  'дней': 'O',
  'успел': 'O',
  'принять': 'O',
  'у': 'O',
  'лидеров': 'O',
  'Украины,': 'O',
  'Польши': 'O',
  'Чехии': 'O'},
 {'состоит': 'B-cV',
  'часть их': 'B-cR1',
  'одного': 'B-cR2',
  '': 'V',
  'часть': 'B-R1',
  'из': 'B-R2',
  'двух': 'I-R2',
  'Работа': 'O',
  'с': 'O',
  'двухбайтовыми': 'O',
  'наборами': 'O',
  'символов': 'O',
  '—': 'O',
  'просто': 'O',
  'кошмар': 'O',
  'для': 'O',
  'программиста,': 'O',
  'так': 'O',
  'как': 'O',
  'байта,': 'O',
  'а': 'O',
  '.': 'O'},
 {'Заместитель': 'O',
  'Генерального': 'O',
  'секретаря': 'O',
  'подчеркнул,': 'O',
  'что': 'O',
  'проблема': 'O',
  'наркотиков': 'O',
  'не': 'O',
  'только': 'O',
  'дестабилизирует': 'O',
  'обстановку': 'O',
  'в': 'O',
  'стране,': 'O',
  'но'

In [87]:
def process_encoded(encoded):
    sentences = []
    IOBs = []
    
    ru = ' йцукенгшщзхъфывапролджэячсмитьбюё'
    for sent in encoded:
        words_list = []
        IOB_list = []
        for phrase in sent:
            IOB_current = sent[phrase]
            phrase = ''.join([letter for letter in phrase if letter in ru])
            words = word_tokenize(phrase)
            leng = words
            for word in words:
                words_list.append(word)
                IOB_list.append(IOB_current)
            
        sentences.append(words_list)
        IOBs.append(IOB_list)
    
    
    
    return sentences, IOBs

train_sent, train_IOB = process_encoded(train_encoded)

In [94]:
def make_IOB_index(encoded):
    '''
    Create a vocabulary for label indexing.
    '''
    IOB_index = {}
    n = 0
    for i in encoded:
        for j in i.values():
            if j not in IOB_index:
                IOB_index[j] = n
                n += 1
    
    return IOB_index

IOB_index = make_IOB_index(train_encoded)

In [183]:
import numpy as np
def labels_IOB(IOB, MAX_length, IOB_index):
    '''
    Create a list of lists representing the IOB properties of each word.
    '''
    labels = []
    for i in range(len(IOB)):
        add = [0]*MAX_length
        for j in range(len(IOB[i])):
            add[j] = IOB_index[IOB[i][j]]
        labels.append(add)
        
    return labels

IOB_train_labels = labels_IOB(train_IOB, 50, IOB_index)

Now we build the model.

In [176]:
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(InputLayer(input_shape=(MAX_length, )))
model.add(Embedding(vocab_size, 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(IOB_index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 50, 128)           8142848   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 512)           788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 50, 9)             4617      
_________________________________________________________________
activation_2 (Activation)    (None, 50, 9)             0         
Total params: 8,935,945
Trainable params: 8,935,945
Non-trainable params: 0
_________________________________________________________________


In [185]:
def to_categorical(sequences, categories):
    '''
    Turn the index-encoding for labels into categorical-encoding.
    '''
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [258]:
cat_train_tags_y = to_categorical(IOB_train_labels, len(IOB_index))

In [189]:
model.fit(padded_docs, cat_train_tags_y, epochs=50, batch_size=128, validation_split=0.2)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 12716 samples, validate on 3180 samples
Epoch 1/50
12716/12716 [==============================] - 60s 5ms/step - loss: 0.3972 - acc: 0.9200 - val_loss: 0.2671 - val_acc: 0.9267
Epoch 2/50
12716/12716 [==============================] - 58s 5ms/step - loss: 0.2235 - acc: 0.9311 - val_loss: 0.2483 - val_acc: 0.9243
Epoch 3/50
12716/12716 [==============================] - 59s 5ms/step - loss: 0.1547 - acc: 0.9461 - val_loss: 0.2668 - val_acc: 0.9307
Epoch 4/50
12716/12716 [==============================] - 59s 5ms/step - loss: 0.1220 - acc: 0.9573 - val_loss: 0.2838 - val_acc: 0.9321
Epoch 5/50
12716/12716 [==============================] - 60s 5ms/step - loss: 0.1042 - acc: 0.9640 - val_loss: 0.3289 - val_acc: 0.9275
Epoch 6/50
12716/12716 [==============================] - 60s 5ms/step - loss: 0.0921 - acc: 0.9676 - val_loss: 0.3537 - val_acc: 0.9286
Epoch 7/50
12716/12716 [===========================

In [195]:
model.save('IOB.model')

In [216]:
model = load_model('IOB.model')

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


# 8. Оценка результатов полной разметки

In [217]:
test_encoded = IOB_encoder(test)

In [218]:
to_filter = [test_encoded[i] for i in range(len(test))if i not in text_index_test]
for sent in to_filter:
    test_encoded.remove(sent)

In [219]:
test_sent, test_IOB = process_encoded(test_encoded)

NameError: name 'process_encoded' is not defined

In [199]:
IOB_test_labels = labels_IOB(test_IOB, 50, IOB_index)

In [194]:
cat_test_tags_y = to_categorical(IOB_test_labels, len(IOB_index))

In [201]:
scores = model.evaluate(padded_test, cat_test_tags_y)
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

1994/1994 [==============================] - 3s 1ms/step
acc: 92.1985959337133


The achieved accuracy is 92.2%. The next step is to make a prediction.

In [224]:
predictions = model.predict(padded_test)

In [227]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [229]:
pred_results = logits_to_tokens(predictions, {i: t for t, i in IOB_index.items()})

In [262]:
index = 1

In [270]:
print('The original word bag:', test_sent[index])
print('The original IOB properties:', test_IOB[index])
print('The predicted IOB:', pred_results[index])


The original word bag: ['называют', 'ти', 'состояния', 'фазами', 'воды', 'превращения', 'из', 'одного', 'состояния', 'в', 'другое', 'фазовыми', 'переходами', 'а']
The original IOB properties: ['B-cV', 'B-cR1', 'B-cR1', 'B-cR2', 'I-cR2', 'B-R1', 'B-R1', 'B-R1', 'B-R1', 'B-R1', 'B-R1', 'B-R2', 'I-R2', 'O']
The predicted IOB: ['B-cV', 'B-cR1', 'B-cR1', 'I-cR2', 'B-R1', 'B-R1', 'B-R1', 'I-R2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


We can see that the sequence of prediction is fairly similar. If one word is predicted wrong, the chain effect will probably make the predictions for the following words wrong.

# 9. Анализ результатов